In [ ]:
project_path = "/home/jupyter"
import os
import sys
sys.path.append(project_path)
from google.cloud import bigquery, storage

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px

from fintrans_toolbox.src import bq_utils as bq

In [ ]:
client = bigquery.Client()

In [ ]:
# Summarise the data by country
UK_spending_by_country = '''SELECT time_period_value, destination_country, SUM(spend) AS Total FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` where time_period = 'Quarter' and mcg = 'All' and merchant_channel = 'Online' and cardholder_origin_country = 'All' and cardholder_origin = 'UNITED KINGDOM' and destination_country != 'UNITED KINGDOM' GROUP BY destination_country, 
time_period_value, spend ORDER BY time_period_value, destination_country DESC'''
df_by_country = bq.read_bq_table_sql(client, UK_spending_by_country)
df_by_country.head()


In [ ]:
# Summarise the data by country
UK_spending_by_country = '''SELECT time_period_value, destination_country, SUM(spend) AS Total FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` where time_period = 'Quarter' and mcg = 'All' and merchant_channel = 'Online' and cardholder_origin_country = 'All' and cardholder_origin = 'UNITED KINGDOM' and destination_country != 'UNITED KINGDOM' GROUP BY destination_country, 
time_period_value ORDER BY time_period_value, destination_country DESC'''
df_by_country = bq.read_bq_table_sql(client, UK_spending_by_country)
df_by_country.head()


In [ ]:
# Save the DataFrame to a CSV file
df_by_country.to_csv('df_by_country.csv', index=False)

In [ ]:
import pandas as pd

# Assuming df_by_country is the DataFrame with your data
# Ensure 'time_period_value' is a string type and split it to get the year (assuming 'Q1', 'Q2', etc., are part of the time_period_value)

# Extract the year from the time_period_value (assuming it's in the format like '2023-Q1', '2023-Q2', etc.)
df_by_country['year'] = df_by_country['time_period_value'].str[:4].astype(int)

# Now group by year and sum the spend for each year
df_yearly_spend = df_by_country.groupby('year')['Total'].sum().reset_index()

# Optionally, you can sort the result by year
df_yearly_spend = df_yearly_spend.sort_values(by='year')

# Display the yearly totals
print(df_yearly_spend)

In [ ]:
import pandas as pd

# Assuming df_by_country is the DataFrame with the data from the SQL query

# Extract the year from the 'time_period_value' (assuming it's in format like '2023-Q1', '2023-Q2', etc.)
df_by_country['year'] = df_by_country['time_period_value'].str[:4].astype(int)

# Filter the data for the years 2019 to 2024
df_filtered = df_by_country[df_by_country['year'].between(2019, 2024)]

# Group by year and sum the total spend for each year
df_yearly_spend = df_filtered.groupby('year')['Total'].sum().reset_index()

# Sort the result by year
df_yearly_spend = df_yearly_spend.sort_values(by='year')

# Display the yearly totals
print(df_yearly_spend)


In [ ]:
import pandas as pd

# Assuming df_by_country is the DataFrame with your data
# Extract the year from the time_period_value (assuming it's in the format like '2023-Q1', '2023-Q2', etc.)
df_by_country['year'] = df_by_country['time_period_value'].str[:4].astype(int)

# Filter data for the years 2019 to 2024
df_filtered = df_by_country[df_by_country['year'].between(2019, 2024)]

# Group by year and sum the spend for each year
df_yearly_spend = df_filtered.groupby('year')['spend'].sum().reset_index()

# Sort the result by year
df_yearly_spend = df_yearly_spend.sort_values(by='year')

# Display the yearly totals
print(df_yearly_spend)


In [ ]:
# Assign value and percentage change variables
df_by_country = df_by_country.sort_values(['destination_country','time_period_value'])
df_by_country['pct_change'] = df_by_country.groupby(['destination_country']).spend.pct_change()
df_by_country['value_change'] = df_by_country.groupby(['destination_country']).spend.diff()


In [ ]:
# Plot stacked bar chart with all values
df_by_country[['time_period_value','destination_country','spend']].groupby(['time_period_value','destination_country']).sum().unstack().plot(kind = 'bar', stacked = 'True')

In [ ]:
# Plot line chart using plotly express for all countries
pfig2 = px.line(
        df_by_country,
        x="time_period_value",
        y="spend",
        color="destination_country")
pfig2

In [ ]:
# Select some contries to plot and reduce dataframe
countries = ['AUSTRALIA','SPAIN','FRANCE']
plot_df = df_by_country[['time_period_value','destination_country','spend']].loc[df_by_country['destination_country'].isin(countries) ]

In [ ]:
#stacked bar chart using pandas plot
plot_df.groupby(['time_period_value','destination_country']).sum().unstack().plot(kind = 'bar', stacked = 'True')

In [ ]:
# Line chart using plotly express
pfig = px.line(
        plot_df,
        x="time_period_value",
        y="spend",
        color="destination_country")
pfig

In [ ]:
# Define as EU or non-EU
EU_definition =  [ 'FRANCE','SPAIN','GERMANY','AUSTRIA']
#df_by_country['EU'] = np.where(df_by_country.isin('EU_definition')
                               
                               
df_by_country.loc[df_by_country['destination_country'].isin(EU_definition), 'EU'] = 'EU'
df_by_country.loc[df_by_country['destination_country'].isin(EU_definition) == False, 'EU'] = 'Not EU'
df_by_country

In [ ]:
df_by_country.loc[df_by_country['destination_country'].isin(EU_definition), 'EU'] = 'EU'